In [1]:
import pandas as pd
import datetime

**Metadata**

In [ ]:
!wget https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/metaData.csv

In [5]:
trips = (pd.read_csv('/content/metaData.csv',
                     usecols=[0, 1, 2, 6, 7, 8, 9, 13],
                     dtype={
                         'name': 'string',
                         'busNumber': 'Int16',
                         'startTime_iso': 'string',
                         'drivenDistance': 'float32',
                         'busRoute': 'string',
                         'energyConsumption': 'float32',
                         'itcs_numberOfPassengers_mean': 'float32',
                         'temperature_ambient_mean': 'float32',
                         },
                     na_values={'busRoute': '-'})#gestione NULL
         .convert_dtypes())

**ZTBus dataset**

In [7]:
!wget https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/ZTBus_compressed.zip
!unzip /content/ZTBus_compressed.zip

''

In [6]:
dtype_dict = {
    'gnss_altitude': 'float32',
    'itcs_busRoute': 'string',
    'itcs_numberOfPassengers': 'float32',
    'itcs_stopName': 'string',
    'odometry_vehicleSpeed': 'float32',
    'status_haltBrakeIsActive': 'boolean',
    'status_parkBrakeIsActive': 'boolean',
    'temperature_ambient': 'float32'
} # come un elemento deve essere interpretato
details_dict = {}
for n in trips.name:
    df_temp = pd.read_csv(n + '.csv',
                          usecols=[0, 3, 7, 8, 9, 12, 21, 22, 23],
                          dtype=dtype_dict,
                          na_values={'itcs_busRoute': '-', 'itcs_stopName': '-'}, # gestione NULL
                          parse_dates=['time_iso']) # gestione date
    df_temp['name'] = n
    df_temp['name_short'] = n[:4] # riduzione ai primi 4 caratteri
    details_dict[n] = df_temp
details = pd.concat(details_dict.values(), ignore_index=True)

**0. Transformation of some variables**


Transformation of trips

In [15]:
trips['energyConsumptionKJ']= trips['energyConsumption']/1000 # conversione da unità internazionale joule
trips['celsius.mean'] = round(trips['temperature_ambient_mean'] - 273.15, 3) # K to C
trips["date"] = pd.to_datetime(trips['startTime_iso'])
trips['month'] = trips['date'].dt.month

Transformation of details

In [16]:
details['temperature_celsius']=round(details['temperature_ambient']-273.15,3) # K to C
details['year']=details['time_iso'].dt.year
details['month']=details['time_iso'].dt.month

**1. Extract all trips with busRoute 83**


In [23]:
trips[trips['busRoute']=='83']

,name,busNumber,startTime_iso,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,temperature_ambient_mean,energyConsumptionKJ,celsius.mean,date,month
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,2020-03-03T04:42:38Z,225047.90625,83,1544278016,23.475309,280.545013,1544278.016,7.395,2020-03-03 04:42:38+00:00,3
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,2020-03-06T04:53:23Z,224512.296875,83,1631815936,17.415779,279.88501,1631815.936,6.735,2020-03-06 04:53:23+00:00,3
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,2020-03-09T14:16:13Z,77824.359375,83,540601280,23.18182,281.048889,540601.28,7.899,2020-03-09 14:16:13+00:00,3
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,2020-03-10T04:50:03Z,225095.796875,83,1692171008,20.9641,279.836304,1692171.008,6.686,2020-03-10 04:50:03+00:00,3
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,2020-03-12T04:56:41Z,224181.203125,83,1145859968,17.212351,287.344513,1145859.968,14.195,2020-03-12 04:56:41+00:00,3
...,...,...,...,...,...,...,...,...,...,...,...,...
1399,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,2022-11-30T04:47:53Z,223165.0,83,1560888064,27.890659,280.694794,1560888.064,7.545,2022-11-30 04:47:53+00:00,11
1400,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,2022-12-01T05:19:41Z,190196.0,83,1418846976,26.03927,279.765503,1418846.976,6.616,2022-12-01 05:19:41+00:00,12
1401,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,2022-12-02T04:47:48Z,224473.40625,83,1611149952,24.803841,279.788788,1611149.952,6.639,2022-12-02 04:47:48+00:00,12
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,210041.59375,83,1536696960,28.785391,279.52829,1536696.96,6.378,2022-12-07 05:13:02+00:00,12


**2. Extract all trips where busRoute is not a number**

In [24]:
trips['busRoute'].unique() # valori acquisibili

<StringArray>
[<NA>, '31', '33', '72', '46', '32', '83', 'N4', 'N2', 'N1']
Length: 10, dtype: string

In [25]:
trips[(trips['busRoute']=='N4') | (trips['busRoute']=='N2') | (trips['busRoute']=='N1')]
#trips[trips['busRoute'].isin(['N4', 'N2', 'N1'])]

,name,busNumber,startTime_iso,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,temperature_ambient_mean,energyConsumptionKJ,celsius.mean,date,month
533,B183_2021-12-18_23-37-00_2021-12-19_03-38-35,183,2021-12-18T23:37:00Z,76216.0625,N4,481350304,9.198582,276.86319,481350.304,3.713,2021-12-18 23:37:00+00:00,12
553,B183_2022-01-07_23-40-43_2022-01-08_03-31-21,183,2022-01-07T23:40:43Z,68557.0625,N2,453625088,4.626984,276.967285,453625.088,3.817,2022-01-07 23:40:43+00:00,1
554,B183_2022-01-08_23-40-17_2022-01-09_03-35-32,183,2022-01-08T23:40:17Z,67962.921875,N2,475383296,7.495495,278.564514,475383.296,5.415,2022-01-08 23:40:17+00:00,1
561,B183_2022-01-15_23-41-46_2022-01-16_03-40-23,183,2022-01-15T23:41:46Z,77156.703125,N1,525168288,6.5125,274.993713,525168.288,1.844,2022-01-15 23:41:46+00:00,1
568,B183_2022-01-21_23-35-40_2022-01-22_03-26-24,183,2022-01-21T23:35:40Z,71917.75,N2,455476000,5.357143,275.307312,455476.0,2.157,2022-01-21 23:35:40+00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1373,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,2022-10-21T22:38:32Z,78567.15625,N1,434776608,16.33333,289.255005,434776.608,16.105,2022-10-21 22:38:32+00:00,10
1374,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,2022-10-22T22:34:45Z,73427.96875,N2,399773696,17.710529,287.348602,399773.696,14.199,2022-10-22 22:34:45+00:00,10
1394,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,2022-11-25T23:35:16Z,72911.257812,N2,447553408,11.21667,281.388397,447553.408,8.238,2022-11-25 23:35:16+00:00,11
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,59548.570312,N1,451916512,20.105261,279.45401,451916.512,6.304,2022-12-09 23:55:12+00:00,12


**3. For each (busNumber, busRoute) pair, determine the number of trips**

In [26]:
trips.groupby(['busNumber', 'busRoute']).size()

busNumber  busRoute
183        31           12
           32           12
           33          130
           46          104
           72          114
           83          441
           N1           10
           N2           19
           N4           11
208        31            5
           32           14
           33           25
           46           19
           72           44
           83          405
           N1            6
           N2           20
           N4            7
dtype: int64

**4. For each trip, compute the ratio between the energy consumption and the average number of passengers**

In [27]:
trips['energyConsumptionKJ'] / trips['itcs_numberOfPassengers_mean']

0        86405.005626
1        12147.474535
2         73427.93888
3       167332.782709
4        26131.895519
            ...      
1404     10702.152526
1405      53384.61332
1406     47380.468015
1407     22477.525429
1408     33457.694889
Length: 1409, dtype: Float64

**5. For each station (itcs_stopName), determine the average number of passengers.**

In [28]:
details.groupby(['itcs_stopName'])[['itcs_numberOfPassengers']].mean()

,itcs_numberOfPassengers
itcs_stopName,
"Zürich, Albisrank",16.196598
"Zürich, Albisriederplatz",25.453398
"Zürich, Altes Krematorium",21.980349
"Zürich, Bahnhof Affoltern",2.850746
"Zürich, Bahnhof Altstetten",7.816580
...,...
"Zürich, Zentrum Witikon",17.255102
"Zürich, Zweiackerstrasse",7.502203
"Zürich, Zwielplatz",2.049383


**6. For each station, determine the buses that have stopped there at least once.**

In [10]:
stopped = details.groupby(['itcs_stopName', 'name_short']).size()
stopped[stopped>= 1]

itcs_stopName                   name_short
Zürich, Albisrank               B183           6324
                                B208           6199
Zürich, Albisriederplatz        B183          11218
                                B208           7633
Zürich, Altes Krematorium       B183          10022
                                              ...  
Zürich, Zwielplatz              B208              4
Zürich, Zwinglihaus             B183           2791
                                B208           1231
Zürich,Kalkbreite/Bhf.Wiedikon  B183            279
                                B208            256
Length: 297, dtype: int64

**7. For each station, determine the buses that have stopped there at least ten times.**

In [11]:
stopped[stopped >= 10]

itcs_stopName                   name_short
Zürich, Albisrank               B183           6324
                                B208           6199
Zürich, Albisriederplatz        B183          11218
                                B208           7633
Zürich, Altes Krematorium       B183          10022
                                              ...  
Zürich, Zwielplatz              B183             77
Zürich, Zwinglihaus             B183           2791
                                B208           1231
Zürich,Kalkbreite/Bhf.Wiedikon  B183            279
                                B208            256
Length: 295, dtype: int64

**8. For each (route, bus) pair, compute the ratio between the overall energy consumption and the overall driven distance.**

In [17]:
overall=(trips.groupby(['busRoute', 'busNumber'])[['drivenDistance', 'energyConsumptionKJ']].sum().apply(lambda x: x.energyConsumptionKJ / x.drivenDistance, axis=1))
overall

busRoute  busNumber
31        183          6.086892
          208          5.290863
32        183          6.174586
          208          5.491248
33        183          5.970747
          208          5.639244
46        183          5.631036
          208          5.583111
72        183          5.899030
          208          5.410319
83        183          5.844977
          208          5.819639
N1        183          5.980921
          208          5.640062
N2        183          5.701850
          208          5.405153
N4        183          6.154987
          208          6.067190
dtype: float64

**9. Starting from the results of the previous point, for each route compute the buses with max and min energy ratio, and save the difference between these ratios in a dataframe.**

In [18]:
max_min = (overall.groupby('busRoute').agg(bus_max_ratio=lambda x: x.idxmax()[1],
                                          bus_min_ratio=lambda x: x.idxmin()[1],
                                          difference=lambda x: x.max() - x.min()))
max_min

,bus_max_ratio,bus_min_ratio,difference
busRoute,,,
31,183,208,0.796029
32,183,208,0.683338
33,183,208,0.331503
46,183,208,0.047925
72,183,208,0.488711
83,183,208,0.025338
N1,183,208,0.340859
N2,183,208,0.296697
N4,183,208,0.087797


**10. Find the bus maximizing the difference computed in the previous point.**

In [21]:
max_min.loc[max_min['difference'].idxmax()]

bus_max_ratio    183.000000
bus_min_ratio    208.000000
difference         0.796029
Name: 31, dtype: float64

**11. Extract the rows of the details such that the gnss_altitude differs from the value in the preceding row. Store also the difference in the variable altitude_variation.**

In [36]:
details['altitude_variation'] = details['gnss_altitude'].diff()
details[(details['altitude_variation'] != 0) & (details['altitude_variation'].notnull())]

,time_iso,gnss_altitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name,name_short,altitude_variation
148,2019-04-30 03:21:24+00:00,397.829895,<NA>,NaN,<NA>,3.793694,False,False,292.149994,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,B183,0.029907
149,2019-04-30 03:21:25+00:00,400.313995,<NA>,NaN,<NA>,4.045902,False,False,292.149994,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,B183,2.484100
150,2019-04-30 03:21:26+00:00,401.689209,<NA>,NaN,<NA>,4.404461,False,False,292.149994,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,B183,1.375214
151,2019-04-30 03:21:27+00:00,400.485992,<NA>,NaN,<NA>,4.860109,False,False,292.149994,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,B183,-1.203217
152,2019-04-30 03:21:28+00:00,399.119202,<NA>,NaN,<NA>,5.394077,False,False,292.149994,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,B183,-1.366791
...,...,...,...,...,...,...,...,...,...,...,...,...
48674431,2022-12-11 03:28:35+00:00,416.991302,<NA>,NaN,<NA>,0.000000,True,True,278.149994,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,B208,0.171814
48674432,2022-12-11 03:28:36+00:00,417.191193,<NA>,NaN,<NA>,0.000000,True,True,278.149994,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,B208,0.199890
48674433,2022-12-11 03:28:37+00:00,416.053986,<NA>,NaN,<NA>,0.000000,True,True,278.149994,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,B208,-1.137207
48674457,2022-12-11 03:29:01+00:00,410.191589,<NA>,NaN,<NA>,0.000000,False,True,278.149994,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,B208,0.088776


**12. For each details dataset, compute the sum of the absolute value (i.e. the sign is not considered) of altitude_variation.**

In [37]:
details.groupby('name')['altitude_variation'].apply(lambda x: abs(x).sum())

name
B183_2019-04-30_03-18-56_2019-04-30_08-44-20      184.739075
B183_2019-04-30_13-22-07_2019-04-30_17-54-02     3896.828125
B183_2019-05-01_05-58-51_2019-05-01_22-32-30      637.407471
B183_2019-05-03_02-50-21_2019-05-03_05-53-20     4074.479004
B183_2019-05-03_15-41-57_2019-05-03_23-06-24    13325.948242
                                                    ...     
B208_2022-12-06_14-43-49_2022-12-06_18-22-52     2300.395752
B208_2022-12-07_05-13-02_2022-12-07_19-19-53     8617.676758
B208_2022-12-08_05-22-20_2022-12-08_18-39-15     8022.732422
B208_2022-12-09_23-55-12_2022-12-10_03-24-28     2748.422119
B208_2022-12-10_23-34-46_2022-12-11_03-29-05     3247.223633
Name: altitude_variation, Length: 1409, dtype: float32

**13. For each month of the year, compute the average ambient temperature**

In [12]:
details.groupby(['year', 'month'])['temperature_celsius'].mean()

year  month
2019  4        11.592139
      5        14.262241
      6        24.686572
      7        24.222378
      8        21.506830
      9        16.888168
      10       14.333965
      11        7.942168
      12        6.471598
2020  1         6.347809
      2         6.978495
      3         8.745139
      4        16.253150
      5        16.572134
      7        24.661377
      8        24.345488
      9        21.964239
      10       11.601410
      11        8.284370
      12        5.535946
2021  1         3.556707
      2         9.243736
      3         9.181781
      4        12.957010
      5        15.510111
      6        23.096718
      7        22.862227
      8        21.107208
      9        21.014938
      10       13.272883
      11        7.463877
      12        6.967660
2022  1         5.545923
      2         7.837626
      3        11.547950
      4        13.882865
      5        20.479652
      6        23.701928
      7        26.040652
      8      

In [44]:
trips.groupby("month")["celsius.mean"].mean()

month
1           5.8
2         7.375
3     10.138614
4     13.465517
5     16.920246
6     23.006897
7         24.24
8     22.723078
9     19.207144
10        14.55
11     8.721312
12     6.235294
Name: celsius.mean, dtype: Float32

**14. For each bus compute the total time when the halt brake is active and the total time when the park brake is active. Compute also the ratio between those two times.**

In [22]:
(details.groupby('name_short')[['status_haltBrakeIsActive', 'status_parkBrakeIsActive']].sum()
.assign(ratio=lambda x: x.status_haltBrakeIsActive / x.status_parkBrakeIsActive))

,status_haltBrakeIsActive,status_parkBrakeIsActive,ratio
name_short,,,
B183,7837532,3184639,2.461043
B208,5942139,1828676,3.249421


**15. For each pair of stops that are consecutive in at least a trip, compute the average speed achieved when going from the first to the second stop.**

In [27]:
aux = {}  #dizionario ausiliare
for n, y in details_dict.items():
    y = y.loc[ y.itcs_stopName.first_valid_index():y.itcs_stopName.last_valid_index()]  # gestione NA
    aux[n] = (
        y.assign(
            pair=list(
                zip(y.itcs_stopName.ffill(), y.itcs_stopName.bfill())
                )
            )
        [['odometry_vehicleSpeed', 'pair']]
        )
x = pd.concat(aux, ignore_index=True).groupby('pair')[['odometry_vehicleSpeed']].mean()
x[x.index.str[0] != x.index.str[1]]

,odometry_vehicleSpeed
pair,
"(Zürich, Albisrank, Zürich, Flurstrasse)",4.385559
"(Zürich, Albisrank, Zürich, Hubertus)",4.148107
"(Zürich, Albisriederplatz, Zürich, Altes Krematorium)",3.521361
"(Zürich, Albisriederplatz, Zürich, Friedhof Sihlfeld)",4.687531
"(Zürich, Albisriederplatz, Zürich, Hardplatz)",4.364703
...,...
"(Zürich, Zwinglihaus, Zürich, Goldbrunnenplatz)",3.632436
"(Zürich, Zwinglihaus, Zürich, Schmiede Wiedikon)",3.204749
"(Zürich, Zwinglihaus, Zürich,Kalkbreite/Bhf.Wiedikon)",3.752148
